In [165]:
import pandas as pd
from selenium import webdriver
import numpy as np
from bs4 import BeautifulSoup
import re
import requests
from time import sleep
import json
import warnings
warnings.filterwarnings('ignore')

In [2]:
dir_for_shinhan = 'C:/Users/shic/Desktop/shinhan_simon/chromedriver/chromedriver.exe'
dir_for_seunghun = 'C:/Users/rsh15/Desktop/seunghuni/selenium/chromedriver.exe'

In [142]:
driver = webdriver.Chrome(dir_for_shinhan)
driver.maximize_window()
driver.refresh()


In [148]:
kb_list = []
base_url = 'https://onland.kbstar.com/quics?page=C060737&%EB%B2%95%EC%A0%95%EB%8F%99%EB%8C%80%EC%A7%80%EC%97%AD%EC%BD%94%EB%93%9C={}&%EA%B8%B0%EC%A4%80%EB%85%84={}&%EA%B8%B0%EC%A4%80%EC%9B%94={}&QSL=F'
for year in range(2015,2022):
    year = str(year)
    for month in range(1,13):
        if int(month) < 10:
            month = '0' + str(month)
        else:
            month = str(month)
        for region in ['9800000000']:
            # 지역, 기준년, 기준월 조합해서 URL format
            tmp_url = base_url.format(region,year,month)
            
            # URL 접속
            driver.get(url = tmp_url) 
            
            # 그래프의 데이터 가져오기
            soup = BeautifulSoup(driver.page_source,'html.parser')
            
            aa = re.search('var graphYn1 = [$](.+?);',driver.page_source,re.S).group(1)
            aa = aa.replace(".trim('[","").replace("') == '' ? false: true","").replace('Date.UTC(','')
            aa = aa.split('], [')
            
            date_list_aa = []
            data_list_aa = []
            for k in aa:
                tmpp = k.split('),')
                data_wrangle = tmpp[0].split(', ')
                data_wrangle[1] = str(int(data_wrangle[1]) + 1)
                
                date_list_aa.append(','.join(data_wrangle))
                data_list_aa.append(float(tmpp[1].replace(']','')))
                
            bb = re.search('var graphYn2 = [$](.+?);',driver.page_source,re.S).group(1)
            bb = bb.replace(".trim('[","").replace("') == '' ? false: true","").replace('Date.UTC(','')
            bb = bb.split('], [')
            
            date_list_bb = []
            data_list_bb = []
            for k in bb:
                tmpp = k.split('),')
                data_wrangle = tmpp[0].split(', ')
                data_wrangle[1] = str(int(data_wrangle[1]) + 1)
                
                date_list_bb.append(','.join(data_wrangle))
                data_list_bb.append(float(tmpp[1].replace(']','').replace(',','')))
                
            kb_list.append(pd.DataFrame({
                'date_a' : pd.to_datetime(date_list_aa,format='%Y,%m,%d'),
                'value1' : data_list_aa,
                'date_b' : pd.to_datetime(date_list_bb,format='%Y,%m,%d'),
                'value2' : data_list_bb
            }))
            
            sleep(2)

In [170]:
df = pd.concat(kb_list,axis=0).drop_duplicates()
df['date_a'] = pd.to_datetime(df['date_a'],format='%Y-%m-%d')
df['date_b'] = pd.to_datetime(df['date_b'],format='%Y-%m-%d')
df = df[['date_a','value1','value2']]

In [171]:
df

,date_a,value1,value2
0,2014-03-03,88.4,0.10
1,2014-03-10,88.4,0.08
2,2014-03-17,88.5,0.07
3,2014-03-24,88.5,0.06
4,2014-03-31,88.6,0.05
...,...,...,...
42,2021-06-07,119.8,0.42
43,2021-06-14,120.2,0.39
44,2021-06-21,120.7,0.36
45,2021-06-28,121.2,0.42
